In [1]:
import datetime

import tensorflow as tf

# Imports for the HParams plugin
from tensorboard.plugins.hparams import api_pb2
from tensorboard.plugins.hparams import summary as hparams_summary
from google.protobuf import struct_pb2

In [10]:
# Get MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

# Multi-layer Perceptrons

## Experiment setup

This experiment is to observe the effect of hypereparameters (listed below) on the accuracy of a shallow neural network.
* Learning rate
* Number of units on the hidden layer
* Mini-batch size
* Drop out rate

In [27]:
learning_rate_list = [0.001, 0.01, 0.1]
num_units_list = [16, 32]
mini_batch_size_list = [32, 64]
dropout_rate_list = [0., 0.5]

Inform the HParams dashboard the hyperparameters and metrics. 

In [28]:
def create_experiment_summary(learning_rate_list, num_units_list, mini_batch_size_list, dropout_rate_list):
    learning_rate_list_val = struct_pb2.ListValue()
    learning_rate_list_val.extend(learning_rate_list)
    
    num_units_list_val = struct_pb2.ListValue()
    num_units_list_val.extend(num_units_list)
    
    mini_batch_size_list_val = struct_pb2.ListValue()
    mini_batch_size_list_val.extend(mini_batch_size_list)
    
    dropout_rate_list_val = struct_pb2.ListValue()
    dropout_rate_list_val.extend(dropout_rate_list)
    
    return hparams_summary.experiment_pb(
      # The hyperparameters being changed
      hparam_infos=[
          api_pb2.HParamInfo(name='learning_rate',
                             display_name='Learning Rate',
                             type=api_pb2.DATA_TYPE_FLOAT64,
                             domain_discrete=learning_rate_list_val),
          api_pb2.HParamInfo(name='num_units',
                             display_name='Number of units',
                             type=api_pb2.DATA_TYPE_FLOAT64,
                             domain_discrete=num_units_list_val),
          api_pb2.HParamInfo(name='mini_batch_size',
                             display_name='Mini Batch Size',
                             type=api_pb2.DATA_TYPE_FLOAT64,
                             domain_discrete=mini_batch_size_list_val),
          api_pb2.HParamInfo(name='dropout_rate',
                             display_name='Dropout rate',
                             type=api_pb2.DATA_TYPE_FLOAT64,
                             domain_discrete=dropout_rate_list_val),
      ],
      # The metrics being tracked
      metric_infos=[
          api_pb2.MetricInfo(
              name=api_pb2.MetricName(
                  tag='accuracy'),
              display_name='Accuracy'),
      ]
    )


exp_summary = create_experiment_summary(learning_rate_list, num_units_list, mini_batch_size_list, dropout_rate_list)
root_logdir_writer = tf.summary.create_file_writer("logs/hparam_tuning")
with root_logdir_writer.as_default():
    tf.summary.import_event(tf.compat.v1.Event(summary=exp_summary).SerializeToString())

## Adapt TensorFlow run to log hyperparameters and metric
The difference compared to regular training routine is the hyperparameters are no longer hardcoded. Instead, they are provided as `hparam` dictionary.

In [29]:
def train_mlp(hparams):

    model = tf.keras.models.Sequential([
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(hparams['num_units'], activation=tf.nn.relu),
                tf.keras.layers.Dropout(hparams['dropout_rate']),
                tf.keras.layers.Dense(10, activation=tf.nn.softmax)
                ])
    
    optim = tf.keras.optimizers.Adam(lr=hparams['learning_rate'])
    
    model.compile(optimizer=optim,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    model.fit(x_train, 
              y_train,
              batch_size=hparams['mini_batch_size'],
              epochs=1) # Run with 1 epoch to speed things up for demo purposes
    
    _, accuracy = model.evaluate(x_test, y_test)
    return accuracy

For each run, log an hparams summary with the hyperparameters and final accuracy

In [30]:
def run(run_dir, hparams):
    writer = tf.summary.create_file_writer(run_dir)
    summary_start = hparams_summary.session_start_pb(hparams=hparams)

    with writer.as_default():
        accuracy = train_mlp(hparams)
        summary_end = hparams_summary.session_end_pb(api_pb2.STATUS_SUCCESS)

        tf.summary.scalar('accuracy', accuracy, step=1, description="The accuracy")
        tf.summary.import_event(tf.compat.v1.Event(summary=summary_start).SerializeToString())
        tf.summary.import_event(tf.compat.v1.Event(summary=summary_end).SerializeToString())

## Start running & log the accuaracy with different hyperparameters

In [31]:
session_num = 0

for learning_rate in learning_rate_list:
    for num_units in num_units_list:
        for mini_batch_size in mini_batch_size_list:
            for dropout_rate in dropout_rate_list:
                hparams = {'learning_rate':learning_rate, 
                           'num_units': num_units, 
                           'mini_batch_size': mini_batch_size,
                           'dropout_rate': dropout_rate
                          }
                print('--- Running training session %d' % (session_num + 1))
                print(hparams)
                run_name = "run-%d" % session_num
                run("logs/hparam_tuning/" + run_name, hparams)
                session_num += 1
      

--- Running training session 1
{'learning_rate': 0.001, 'num_units': 16, 'mini_batch_size': 32, 'dropout_rate': 0.0}
10000/10000 [==============================] - 1s 54us/sample - loss: 0.2789 - accuracy: 0.9188
--- Running training session 2
{'learning_rate': 0.001, 'num_units': 16, 'mini_batch_size': 32, 'dropout_rate': 0.5}
10000/10000 [==============================] - 1s 58us/sample - loss: 0.4029 - accuracy: 0.8974
--- Running training session 3
{'learning_rate': 0.001, 'num_units': 16, 'mini_batch_size': 64, 'dropout_rate': 0.0}
10000/10000 [==============================] - 1s 54us/sample - loss: 0.2901 - accuracy: 0.9193
--- Running training session 4
{'learning_rate': 0.001, 'num_units': 16, 'mini_batch_size': 64, 'dropout_rate': 0.5}
10000/10000 [==============================] - 1s 56us/sample - loss: 0.4708 - accuracy: 0.8896
--- Running training session 5
{'learning_rate': 0.001, 'num_units': 32, 'mini_batch_size': 32, 'dropout_rate': 0.0}
10000/10000 [==================